# Chunking Experiments

This notebook contains code for testing best chunking strategies, libraries and snippets.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

pdf_path = "../../data/cao-pdfs/Cao Bouw en Infra 2025 - 2027.pdf"

TOKENIZER_ENCODING = "cl100k_base"  # For OpenAI models
TOKENIZER_MAX_TOKENS = 8192  # Adjust based on your chosen model

MAX_TOKENS = 8192  # Adjust based on your chosen model
VECTOR_DIM = 1536  # Adjust based on your chosen embeddings model

AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY")  # Ensure this is your Admin Key
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME", "cao-rag-sample")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-10-21")
AZURE_OPENAI_CHAT_MODEL_NAME = os.getenv(
    "AZURE_OPENAI_CHAT_MODEL_NAME"
)
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME = os.getenv(
    "AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME", "text-embedding-3-large"
)  # Using a deployed model named "text-embeddings-3-large
AZURE_OPENAI_EMBEDDING_MODEL_NAME = os.getenv(
    "AZURE_OPENAI_EMBEDDING_MODEL_NAME", "text-embedding-3-large"
)  # Using a deployed model named "text-embeddings-3-large

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")

if torch.cuda.is_available():
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}: {torch.cuda.device_count()} GPU(s)")
else:
    print("CUDA is not available. Using CPU.")
    print(f"{torch.cpu.device_count()} CPU core(s) available")


In [3]:
from pathlib import Path
from docling.document_converter import DocumentConverter

converter = DocumentConverter() 

In [4]:
from docling.datamodel.accelerator_options import AcceleratorDevice, AcceleratorOptions
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
)
from docling.datamodel.settings import settings
from docling.document_converter import DocumentConverter, PdfFormatOption

# Explicitly set the accelerator options
accelerator_options = AcceleratorOptions(
    num_threads=8, device=AcceleratorDevice.CUDA
)

pipeline_options = PdfPipelineOptions()
pipeline_options.accelerator_options = accelerator_options
pipeline_options.do_ocr = False
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = True

converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(
            pipeline_options=pipeline_options,
        )
    }
)

In [5]:
# Convert the document
conversion_result = converter.convert(pdf_path)

2025-09-21 10:24:25,103 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-09-21 10:24:27,835 - INFO - Going to convert document batch...
2025-09-21 10:24:27,837 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 12a1aaae4d2de3c2950eea38387e79f2
2025-09-21 10:24:27,866 - INFO - Loading plugin 'docling_defaults'
2025-09-21 10:24:27,871 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-09-21 10:24:27,897 - INFO - Loading plugin 'docling_defaults'
2025-09-21 10:24:27,907 - INFO - Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-09-21 10:24:28,095 - INFO - Accelerator device: 'cuda:0'
2025-09-21 10:24:30,929 - INFO - Accelerator device: 'cuda:0'
2025-09-21 10:24:31,792 - INFO - Processing document Cao Bouw en Infra 2025 - 2027.pdf
2025-09-21 10:26:15,735 - INFO - Finished converting document Cao Bouw en Infra 2025 - 2027.pdf in 110.67 sec.


In [6]:
output_dir = Path("outputs/02-chunking-experiments")
output_dir.mkdir(parents=True, exist_ok=True)

doc_filename = conversion_result.input.file.stem

In [7]:
print(f"Document has {len(conversion_result.document.pages)} pages and {len(conversion_result.document.tables)} tables.")
# print(f"Document text content:\n{conversion_result.document.export_to_markdown()}...")

Document has 178 pages and 68 tables.


In [8]:
# Export tables
'''
import pandas as pd

for table_ix, table in enumerate(conversion_result.document.tables):
    table_df: pd.DataFrame = table.export_to_dataframe(doc=conversion_result.document)
    print(f"## Table {table_ix}")
    print(table_df.to_markdown())

    # Save the table as CSV
    element_csv_filename = output_dir / f"{doc_filename}-table-{table_ix + 1}.csv"
    table_df.to_csv(element_csv_filename)
'''

'\nimport pandas as pd\n\nfor table_ix, table in enumerate(conversion_result.document.tables):\n    table_df: pd.DataFrame = table.export_to_dataframe(doc=conversion_result.document)\n    print(f"## Table {table_ix}")\n    print(table_df.to_markdown())\n\n    # Save the table as CSV\n    element_csv_filename = output_dir / f"{doc_filename}-table-{table_ix + 1}.csv"\n    table_df.to_csv(element_csv_filename)\n'

## Chunking

We convert the Document into smaller chunks for embedding and indexing. The built-in HierarchicalChunker preserves structure.


In [9]:
from docling.chunking import HierarchicalChunker
import tiktoken
from docling_core.transforms.chunker.tokenizer.openai import OpenAITokenizer

# Initialize tiktoken encoding for OpenAI embedding models
encoding = tiktoken.get_encoding(TOKENIZER_ENCODING)

# Create Docling's OpenAITokenizer wrapper
tokenizer = OpenAITokenizer(tokenizer=encoding, max_tokens=TOKENIZER_MAX_TOKENS)

# Instantiate HierarchicalChunker with tokenizer
chunker = HierarchicalChunker(tokenizer=tokenizer, merge_peers=True)

In [ ]:
from xxhash import xxh64
doc_chunks = list(chunker.chunk(conversion_result.document))

all_chunks = []
for idx, c in enumerate(doc_chunks):
    # Enrich chunks (example: add custom metadata or transform)
    chunk_text = chunker.contextualize(c)

    byte_data = chunk_text.encode('utf-8')
    chunk_index = xxh64(byte_data).hexdigest()

    all_chunks.append((chunk_index, chunk_text))

print(f"Total chunks from PDF: {len(all_chunks)}")

Total chunks from PDF: 872
[('b705bc6d366630f9', '2.2  Basisregeling en kaderregeling\nTabel 2.2  Kenmerken van de basis- en de kaderregeling\n\nNormale arbeidsduur, basisregeling voor alle werknemers = Gemiddeld 40 uur per werkweek en gemiddeld 8 uur per dag.. Normale arbeidsduur, kaderregeling grenzen voor het afwijken van de basisregeling = Gemiddeld 40 uur per werkweek en gemiddeld 8 uur per dag.. , basisregeling voor alle werknemers = Beide gemiddelden gemeten over een periode van 13 weken.. , kaderregeling grenzen voor het afwijken van de basisregeling = Beide gemiddelden gemeten over een periode van maximaal 26 weken (infra: maximaal 52 weken).. Werkweek/ werkdagen, basisregeling voor alle werknemers = - Maandag t/m vrijdag. - 5 dagen.. Werkweek/ werkdagen, kaderregeling grenzen voor het afwijken van de basisregeling = - Maandag t/m zaterdag.'), ('ff32a5b7ae0efb2d', '2.2  Basisregeling en kaderregeling\n, 1 = . , 2 = - Niet meer dan gemiddeld 5 dagen, gemeten over een periode va

### Part 3: Create Azure AI Search Index and Push Chunk Embeddings
We’ll define a vector index in Azure AI Search, then embed each chunk using Azure OpenAI and upload in batches.

In [13]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    HnswAlgorithmConfiguration,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SimpleField,
    VectorSearch,
    VectorSearchProfile,
)

VECTOR_FIELD_NAME = "content_vector"
CONTENT_FIELD_NAME = "content"

index_client = SearchIndexClient(
    AZURE_SEARCH_ENDPOINT, AzureKeyCredential(AZURE_SEARCH_API_KEY)
)

def create_search_index(index_name: str):
    # Define fields
    fields = [
        SimpleField(name="chunk_id", type=SearchFieldDataType.String, key=True),
        SearchableField(name=CONTENT_FIELD_NAME, type=SearchFieldDataType.String),
        SearchField(
            name=VECTOR_FIELD_NAME,
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            filterable=False,
            sortable=False,
            facetable=False,
            vector_search_dimensions=VECTOR_DIM,
            vector_search_profile_name="default",
        ),
    ]
    # Vector search config with an AzureOpenAIVectorizer
    vector_search = VectorSearch(
        algorithms=[HnswAlgorithmConfiguration(name="default")],
        profiles=[
            VectorSearchProfile(
                name="default",
                algorithm_configuration_name="default",
                vectorizer_name="default",
            )
        ],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="default",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=AZURE_OPENAI_ENDPOINT,
                    deployment_name=AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME,
                    model_name=AZURE_OPENAI_EMBEDDING_MODEL_NAME,
                    api_key=AZURE_OPENAI_API_KEY,
                ),
            )
        ],
    )

    # Create or update the index
    new_index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)
    try:
        index_client.delete_index(index_name)
    except Exception:
        pass

    index_client.create_or_update_index(new_index)
    print(f"Index '{index_name}' created.")

In [14]:
create_search_index(AZURE_SEARCH_INDEX_NAME)

2025-09-21 10:28:34,896 - INFO - Request URL: 'https://doc-intel-ais.search.windows.net/indexes('cao-rag-sample')?api-version=REDACTED'
Request method: 'DELETE'
Request headers:
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=minimal'
    'x-ms-client-request-id': 'f6eccca6-96c4-11f0-877e-28a44ac6879c'
    'User-Agent': 'azsdk-python-search-documents/11.5.3 Python/3.13.3 (Windows-11-10.0.26100-SP0)'
No body was attached to the request
2025-09-21 10:28:35,929 - INFO - Response status: 404
Response headers:
    'Cache-Control': 'no-cache,no-store'
    'Pragma': 'no-cache'
    'Content-Length': '282'
    'Content-Type': 'application/json; charset=utf-8'
    'Content-Language': 'REDACTED'
    'Expires': '-1'
    'request-id': 'f6eccca6-96c4-11f0-877e-28a44ac6879c'
    'elapsed-time': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'
    'Date': 'Sun, 21 Sep 2025 08:28:36 GMT'
2025-09-21 10:28:35,933 - INFO - Request URL: 'https://doc-intel-ais.search.windows.n

Index 'cao-rag-sample' created.


### Generate Embeddings and Upload to Azure AI Search

In [15]:
from azure.search.documents import SearchClient
from openai import AzureOpenAI

search_client = SearchClient(
    AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_INDEX_NAME, AzureKeyCredential(AZURE_SEARCH_API_KEY)
)
openai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)


def embed_text(text: str):
    """
    Helper to generate embeddings with Azure OpenAI.
    """
    response = openai_client.embeddings.create(
        input=text, model=AZURE_OPENAI_EMBEDDING_MODEL_NAME, dimensions=VECTOR_DIM
    )
    return response.data[0].embedding

In [ ]:
upload_docs = []
for chunk_id, chunk_text in all_chunks:
    embedding_vector = embed_text(chunk_text)
    upload_docs.append(
        {
            "chunk_id": chunk_id,
            "content": chunk_text,
            "content_vector": embedding_vector,
        }
    )

2025-09-21 10:28:39,983 - INFO - HTTP Request: POST https://cao-intel-ai-services.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-09-21 10:28:40,169 - INFO - HTTP Request: POST https://cao-intel-ai-services.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-09-21 10:28:40,295 - INFO - HTTP Request: POST https://cao-intel-ai-services.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-09-21 10:28:40,403 - INFO - HTTP Request: POST https://cao-intel-ai-services.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-09-21 10:28:40,512 - INFO - HTTP Request: POST https://cao-intel-ai-services.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200

Uploaded batch 0 -> 50; all_succeeded: True, first_doc_status_code: 201


2025-09-21 10:30:30,866 - INFO - Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=none; odata.streaming=true; charset=utf-8'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'
    'Preference-Applied': 'REDACTED'
    'OData-Version': 'REDACTED'
    'request-id': '3aeb0811-96c5-11f0-85d5-28a44ac6879c'
    'elapsed-time': 'REDACTED'
    'Date': 'Sun, 21 Sep 2025 08:30:30 GMT'
2025-09-21 10:30:30,936 - INFO - Request URL: 'https://doc-intel-ais.search.windows.net/indexes('cao-rag-sample')/docs/search.index?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '1746563'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': '3c1729e8-96c5-11f0-9d86-28a44ac6879c'
    'User-Agent': 'azsdk-python-search-documents/11.5.3 Python/3.13.3 (Windows-1

Uploaded batch 50 -> 100; all_succeeded: True, first_doc_status_code: 201


2025-09-21 10:30:33,379 - INFO - Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=none; odata.streaming=true; charset=utf-8'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'
    'Preference-Applied': 'REDACTED'
    'OData-Version': 'REDACTED'
    'request-id': '3c1729e8-96c5-11f0-9d86-28a44ac6879c'
    'elapsed-time': 'REDACTED'
    'Date': 'Sun, 21 Sep 2025 08:30:33 GMT'
2025-09-21 10:30:33,459 - INFO - Request URL: 'https://doc-intel-ais.search.windows.net/indexes('cao-rag-sample')/docs/search.index?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '1744376'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': '3d98371e-96c5-11f0-9004-28a44ac6879c'
    'User-Agent': 'azsdk-python-search-documents/11.5.3 Python/3.13.3 (Windows-1

...

Uploaded batch 750 -> 800; all_succeeded: True, first_doc_status_code: 201


2025-09-21 10:30:47,101 - INFO - Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=none; odata.streaming=true; charset=utf-8'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'
    'Preference-Applied': 'REDACTED'
    'OData-Version': 'REDACTED'
    'request-id': '4519a197-96c5-11f0-8d0e-28a44ac6879c'
    'elapsed-time': 'REDACTED'
    'Date': 'Sun, 21 Sep 2025 08:30:47 GMT'
2025-09-21 10:30:47,146 - INFO - Request URL: 'https://doc-intel-ais.search.windows.net/indexes('cao-rag-sample')/docs/search.index?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '765084'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': '45c09c62-96c5-11f0-a557-28a44ac6879c'
    'User-Agent': 'azsdk-python-search-documents/11.5.3 Python/3.13.3 (Windows-11

Uploaded batch 800 -> 850; all_succeeded: True, first_doc_status_code: 201


2025-09-21 10:30:48,023 - INFO - Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=none; odata.streaming=true; charset=utf-8'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'
    'Preference-Applied': 'REDACTED'
    'OData-Version': 'REDACTED'
    'request-id': '45c09c62-96c5-11f0-a557-28a44ac6879c'
    'elapsed-time': 'REDACTED'
    'Date': 'Sun, 21 Sep 2025 08:30:47 GMT'


Uploaded batch 850 -> 872; all_succeeded: True, first_doc_status_code: 201
All chunks uploaded to Azure Search.


In [ ]:
BATCH_SIZE = 50
for i in range(0, len(upload_docs), BATCH_SIZE):
    subset = upload_docs[i : i + BATCH_SIZE]
    resp = search_client.upload_documents(documents=subset)

    all_succeeded = all(r.succeeded for r in resp)
    print(
        f"Uploaded batch {i} -> {i + len(subset)}; all_succeeded: {all_succeeded}, "
        f"first_doc_status_code: {resp[0].status_code}"
    )

print("All chunks uploaded to Azure Search.")

### Part 4: Perform RAG over PDF
Combine retrieval from Azure AI Search with Azure OpenAI Chat Completions (aka. grounding your LLM)

In [17]:
from typing import Optional

from azure.search.documents.models import VectorizableTextQuery

def generate_chat_response(prompt: str, system_message: Optional[str] = None):
    """
    Generates a single-turn chat response using Azure OpenAI Chat.
    If you need multi-turn conversation or follow-up queries, you'll have to
    maintain the messages list externally.
    """
    messages = []
    if system_message:
        messages.append({"role": "system", "content": system_message})
    messages.append({"role": "user", "content": prompt})

    completion = openai_client.chat.completions.create(
        model=AZURE_OPENAI_CHAT_MODEL_NAME, messages=messages, temperature=1
    )
    return completion.choices[0].message.content


user_query = "What are the Probationary period duration of 1-2 year employment contract as per the cao 2025-2027?"
user_embed = embed_text(user_query)

vector_query = VectorizableTextQuery(
    text=user_query,  # passing in text for a hybrid search
    k_nearest_neighbors=5,
    fields=VECTOR_FIELD_NAME,
)

2025-09-21 10:30:48,385 - INFO - HTTP Request: POST https://cao-intel-ai-services.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


In [18]:
search_results = search_client.search(
    search_text=user_query, vector_queries=[vector_query], select=[CONTENT_FIELD_NAME], top=10
)

retrieved_chunks = []
for result in search_results:
    snippet = result[CONTENT_FIELD_NAME]
    retrieved_chunks.append(snippet)

2025-09-21 10:30:48,402 - INFO - Request URL: 'https://doc-intel-ais.search.windows.net/indexes('cao-rag-sample')/docs/search.post.search?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '329'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': '468031c7-96c5-11f0-8638-28a44ac6879c'
    'User-Agent': 'azsdk-python-search-documents/11.5.3 Python/3.13.3 (Windows-11-10.0.26100-SP0)'
A body is sent with the request
2025-09-21 10:30:48,843 - INFO - Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=none; odata.streaming=true; charset=utf-8'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'
    'Preference-Applied': 'REDACTED'
    'OData-Version': 'REDACTED'
    'request-id': '468031c7-96c5-11f0-8638-28a44ac6879c'
    'elapsed-time': 'R

In [19]:
context_str = "\n---\n".join(retrieved_chunks)
rag_prompt = f"""
You are an AI assistant helping answering questions about Dutch CAO.
Use ONLY the text below to answer the user's question.
If the answer isn't in the text, say you don't know.

Context:
{context_str}

Question: {user_query}
Answer:
"""

final_answer = generate_chat_response(rag_prompt)

2025-09-21 10:30:54,989 - INFO - HTTP Request: POST https://cao-intel-ai-services.openai.azure.com/openai/deployments/gpt-5-nano/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


In [20]:
print("\nRAG Prompt and Response:")
print(rag_prompt)


RAG Prompt and Response:

You are an AI assistant helping answering questions about Dutch CAO.
Use ONLY the text below to answer the user's question.
If the answer isn't in the text, say you don't know.

Context:
10.13.1 Looptijd, verlenging, opzegging en vernieuwing
- Deze cao geldt van 1 januari 2025 tot en met 31 maart 2027.
- De cao wordt geacht telkens voor een jaar te zijn verlengd, tenzij een of meer cao-partijen deze hebben opgezegd.
- Voor opzegging van de cao gelden de volgende regels:
- -het moet ten minste drie maanden voorafgaand aan de einddatum gebeuren,
- -bij aangetekend schrijven aan alle cao-partijen.
- Heeft een cao-partij voorstellen ingediend om de cao aan te passen of te vernieuwen? Dan beginnen cao-partijen daar zo snel mogelijk onderhandelingen over.
---
Tabel 1.3.3  Proeftijd
duur arbeidsovereenkomst, 1 = specificatie. duur arbeidsovereenkomst, 2 = maximale proeftijd. duur arbeidsovereenkomst, 3 = maximale proeftijd. , 1 = . , 2 = bouwplaats. , 3 = uta. onbep

In [21]:
print("\nFinal Answer:")
print(final_answer)


Final Answer:
1 maand.
